In [2]:
%pip install boto3

  Using cached boto3-1.42.17-py3-none-any.whl.metadata (6.8 kB)
  Using cached botocore-1.42.17-py3-none-any.whl.metadata (5.9 kB)
  Using cached s3transfer-0.16.0-py3-none-any.whl.metadata (1.7 kB)
Using cached boto3-1.42.17-py3-none-any.whl (140 kB)
Using cached botocore-1.42.17-py3-none-any.whl (14.6 MB)
Using cached s3transfer-0.16.0-py3-none-any.whl (86 kB)
Note: you may need to restart the kernel to use updated packages.


In [9]:


import boto3
import json
from botocore.exceptions import ClientError
temperature = .7


# Set the AWS Region
region = "us-east-1"

# Initialize the Bedrock Runtime client
bedrock = boto3.client("bedrock-runtime", region_name=region)

inference_config = {"temperature": temperature}

MODEL_ID = "amazon.nova-micro-v1:0"


In [10]:
temperature = .7

inference_config = {"temperature": temperature}

system_prompts = [{"text": "You are a virtual travel assistant that suggests destinations based on user preferences."
                + "Only return destination names and a brief description."}]

messages = []

message_1 = {
    "role": "user",
    "content": [{"text": "Create a list of 3 travel destinations."}]
}

messages.append(message_1)

response = bedrock.converse(
    modelId=MODEL_ID,
    messages=messages,
    #system=system_prompts,
    inferenceConfig=inference_config
)

def print_response(response):
    model_response = response.get('output', {}).get('message', {}).get('content', [{}])[0].get('text', '')

    print("✈️ Your suggested travel destinations:")
    print(model_response)

print_response(response)

✈️ Your suggested travel destinations:
Here is a list of 3 travel destinations:

1. Paris, France: Known as the "City of Love," Paris is famous for its iconic landmarks like the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. It is also a hub for art, fashion, gastronomy, and romance.

2. Bali, Indonesia: This beautiful island destination is known for its stunning beaches, luxurious resorts, and rich culture. Visitors can enjoy surfing, temple visits, and exploring the local markets to purchase handmade crafts and souvenirs.

3. Machu Picchu, Peru: This ancient Incan city, nestled in the Andes Mountains, is one of the most famous archaeological sites in the world. Visitors can hike the Inca Trail to reach the site or take a train from nearby Cusco. The breathtaking views and rich history make it a must-visit destination.


In [11]:
message_2 = {
        "role": "user",
        "content": [{"text": "Only suggest travel locations that are no more than one short flight away."}]
}

messages.append(message_2)

response = bedrock.converse(
    modelId=MODEL_ID,
    messages=messages,
    #system=system_prompts,
    inferenceConfig=inference_config
)

print_response(response)

✈️ Your suggested travel destinations:
Sure, here are three travel destinations that are generally reachable with a short flight from many major cities in the United States:

1. **Mexico City, Mexico** – A vibrant metropolis offering rich culture, history, and cuisine, typically accessible with a short flight from cities like Los Angeles, Dallas, or Houston.
   
2. **Cancun, Mexico** – Known for its beautiful beaches and turquoise waters, Cancun is easily reachable with a short flight from cities such as Miami, Atlanta, or New York.

3. **Santo Domingo, Dominican Republic** – The capital city offers a mix of colonial history and modern attractions, and is a short flight away from cities like Miami, New York, and Boston.

These destinations are well-connected via short flights, making them accessible options for a quick getaway.


In [12]:
message_3 = {
        "role": "user",
        "content": [{"text": "De los 6 viajes que indicaste selecciona uno que este mas cerca de argentina"}]
}

messages.append(message_3)

response = bedrock.converse(
    modelId=MODEL_ID,
    messages=messages,
    #system=system_prompts,
    inferenceConfig=inference_config
)

print_response(response)

✈️ Your suggested travel destinations:
Here is a list of three travel destinations that are no more than a short flight away from Argentina:

1. **Uruguay** - Particularly Montevideo, which is just a short flight across the Río de la Plata.
2. **Chile** - Santiago is a bit further but still within a reasonable short flight range.
3. **Brazil** - Buenos Aires has numerous direct flights to cities like São Paulo and Rio de Janeiro.

Among these, **Uruguay** is the closest. Montevideo, the capital of Uruguay, is easily accessible with direct flights from Buenos Aires, making it a convenient and culturally rich destination for travelers from Argentina.


In [13]:
message_3 = {
        "role": "user",
        "content": [{"text": "Habias indicado 6 viajes creo o paises , Mexico, Santo Domingo y otros 6 vaijes puedes indicarme cuales eran ? "}]
}

messages.append(message_3)

response = bedrock.converse(
    modelId=MODEL_ID,
    messages=messages,
    #system=system_prompts,
    inferenceConfig=inference_config
)

print_response(response)

✈️ Your suggested travel destinations:
Claro, aquí tienes una lista de 3 destinos de viaje que están a una corta duración de vuelo desde Argentina:

1. **Buenos Aires, Uruguay** - La ciudad de Montevideo, la capital de Uruguay, está a solo unas dos horas en avión desde Buenos Aires, Argentina. Es un destino ideal para disfrutar de una mezcla de cultura argentina y uruguaya.

2. **São Paulo, Brasil** - São Paulo, una de las ciudades más grandes y vibrantes de América Latina, está aproximadamente a 3 horas en vuelo desde Buenos Aires. Es un destino perfecto para los amantes de la moda, la gastronomía y la vida urbana.

3. **Santiago, Chile** - Santiago, la capital de Chile, se encuentra a alrededor de 2,5 horas en avión desde Buenos Aires. Esta ciudad ofrece una rica mezcla de cultura, historia y moderna vida urbana.

Si necesitas confirmar los otros destinos que mencioné inicialmente, parece que hubo una confusión. Los tres destinos mencionados aquí son los que están a una corta distanc

## Creacion de Solicitud de BedRock Prompt Agent create prompt manager


In [20]:
bedrock_agent = boto3.client("bedrock-agent", region_name=region)

try:
    response = bedrock_agent.create_prompt(
        name="Travel-Agent-Prompt",
        description="Checks if all trip information has been provided.",
        variants=[
            { 
                "name": "Variant1",
                "modelId": MODEL_ID,
                "templateType": "CHAT",
                "inferenceConfiguration": {
                    "text": {
                        "temperature": 0.4
                    }
                },
                "templateConfiguration": { 
                    "chat": {
                        'system': [ 
                            {
                                "text": """You are a travel agent evaluating trip requests for custom itineraries. 
                                Review the message carefully and answer YES or NO to the following screening questions. 
                                Be strict—if any detail is missing or unclear, answer NO.

                                A) Is the destination clearly stated?
                                B) Are the travel dates within a reasonable range (not last−minute or over a year away)?
                                C) Does the request avoid high−risk or restricted activities (e.g., extreme sports, off−grid travel)?
                                D) Is there any mention of a valid passport or travel documentation?
                                E) Is there enough information to follow up with a proposed itinerary?"""
                            }
                        ],
                        'messages': [{
                            'role': 'user',
                            'content': [ 
                                {
                                    'text': "Trip request: {{event_request}}"
                                }
                            ]
                        }],
                        'inputVariables' : [
                            { 'name' : 'event_request'}
                        ]
                    }
                }
        }]
    )
    print("Created!")
    prompt_arn = response.get("arn")
except bedrock.exceptions.ConflictException as e:
    print("Already exists!")
    response = bedrock.list_prompts()
    prompt = next((prompt for prompt in response['promptSummaries'] if prompt['name'] == "TripBooker_xyz"), None)
    prompt_arn = prompt['arn']

#prompt_arn

ConflictException: An error occurred (ConflictException) when calling the CreatePrompt operation: Couldn't perform CreatePrompt operation. The name Travel-Agent-Prompt already exists for id XLU93LP8QE. Retry your request with a different name.

In [ ]:
prompt_arn=''
response = bedrock.converse(
    modelId=prompt_arn,
    promptVariables={
        'event_request': {
            'text': """
                Hi there! I'm planning a trip to Italy with my partner and would love some help organizing the itinerary. We're hoping to travel between September 10–20 this year, ideally flying into Rome and spending a few days in Florence and Venice as well. We’d love recommendations on tours, cultural sites, and good local restaurants. We’re not interested in anything risky like skydiving or hiking remote trails — just want a relaxing and enriching experience. We both have valid passports. Let me know what other details you need!
                """
        }
    },
)
print(response['output']['message']['content'][0]['text'])

YES

Let's break down the trip request based on the screening questions:

A) **Is the destination clearly stated?**
   - Yes, the destination is clearly stated as Italy.

B) **Are the travel dates within a reasonable range (not last−minute or over a year away)?**
   - Yes, the travel dates are within a reasonable range (September 10–20 this year).

C) **Does the request avoid high−risk or restricted activities (e.g., extreme sports, off−grid travel)?**
   - Yes, the request specifies a desire for a relaxing and enriching experience without any mention of high-risk activities.

D) **Is there any mention of a valid passport or travel documentation?**
   - Yes, both travelers have valid passports.

E) **Is there enough information to follow up with a proposed itinerary?**
   - Yes, there is enough information provided to start crafting an itinerary with recommendations on tours, cultural sites, and restaurants in Rome, Florence, and Venice.

Therefore, the answer is YES for all screening 

 ### Tarea 4: Crear una Función Lambda de AWS

Bedrock y Lambda son dos herramientas que puedes emparejar y que se complementan muy bien. Aunque los LLM son muy potentes, no son capaces de hacer todo lo que necesitas que hagan. Bedrock es capaz de llegar a Lambda para llamar a funciones en su nombre. Esto te permite "construir" nuevas funciones en Bedrock.

En esta tarea, crearás una Función Lambda que realiza operaciones matemáticas. En una tarea futura, esto será utilizado por Bedrock.

In [ ]:
import json

def add(num1, num2):
    return num1 + num2

def subtract(num1, num2):
    return num1 − num2

def multiply(num1, num2):
    return num1 * num2

def divide(num1, num2):
    if num2 == 0:
        raise ValueError("division by zero")
    return num1 / num2

def power(num1, num2):
    return num1 ** num2

def lambda_handler(event, context):
    operation = event.get('operation')
    try:
        num1 = float(event['num1'])
        num2 = float(event['num2'])
    except (KeyError, ValueError):
        return {
            'statusCode': 400,
            'body': json.dumps({'error': 'Invalid input, num1 and num2 must be numeric'})
        }

    # Map operations to functions
    operations = {
        'add': add,
        'subtract': subtract,
        'multiply': multiply,
        'divide': divide,
        'power': power
    }

    # Find the operation function and execute
    operation_func = operations.get(operation)
    
    if operation_func:
        try:
            result = operation_func(num1, num2)
            return {
                'statusCode': 200,
                'body': json.dumps({'result': result})
            }
        except ValueError as e:
            return {
                'statusCode': 400,
                'body': json.dumps({'error': str(e)})
            }
    else:
        return {
            'statusCode': 400,
            'body': json.dumps({'error': 'invalid operation'})
        }

In [ ]:
# Setup Bedrock and Lambda
import boto3
import json

bedrock = boto3.client(service_name='bedrock-runtime')
lambda_client = boto3.client("lambda")
MODEL_ID = "amazon.nova-micro-v1:0"

# Define the calculation tool
math_tool = {
    "toolSpec": {
        "name": "calculateNumbers",
        "description": "Performs basic arithmetic operations",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "operation": {"type": "string"},
                    "num1": {"type": "number"},
                    "num2": {"type": "number"}
                },
                "required": ["operation", "num1", "num2"]
            }
        }
    }
}

# Function to trigger the Lambda calculation service
def execute_calculation(input_data):
    response = lambda_client.invoke(
        FunctionName="",  
        InvocationType="RequestResponse",
        Payload=json.dumps(input_data)
    )
    response_payload = response["Payload"].read()
    calculation_result = json.loads(response_payload)
    response_body = calculation_result.get("body", "{}")
    return json.loads(response_body) if isinstance(response_body, str) else response_body

# User's initial message
user_input = {
    "role": "user",
    "content": [{"text": "Please subtract 60 from 100"}]
}

# Define system instructions
system_instructions = [
    {"text": """
    You are a virtual assistant capable of performing basic arithmetic operations: add, subtract, multiply, and divide.
    If the user doesn't specify an operation, ask them for more details.
    """}
]

# First interaction with the model
first_interaction = bedrock.converse(
    modelId=MODEL_ID,
    system=system_instructions,
    messages=[user_input],
    toolConfig={
        "tools": [math_tool],
        "toolChoice": {"auto": {}}
    },
    inferenceConfig={"temperature": 0.7}
)

# Process the assistant's response to check if tool is required
assistant_reply = first_interaction["output"]["message"]
message_parts = assistant_reply["content"]
tool_request_block = next((part for part in message_parts if "toolUse" in part), None)

if not tool_request_block:
    print("=== Assistant's Direct Response ===")
    print(message_parts[0]["text"])
else:
    tool_request = tool_request_block["toolUse"]
    tool_input_data = tool_request["input"]
    tool_id = tool_request["toolUseId"]
    print(tool_request_block)
    print(f"→ Assistant triggered tool: calculateNumbers with input: {tool_input_data}")

    # Execute the requested tool
    tool_result = execute_calculation(tool_input_data)
    print(f"← Lambda Function output: {tool_result}")

    # Create a response based on the tool's output
    try:
        result_summary = f"The outcome of the calculation is {tool_result['result']}."
    except Exception as e:
        result_summary = f"Oops! There was an error with the calculation. ({str(e)})"

    # Generate tool result message
    tool_response_msg = {
        "role": "user",
        "content": [
            {
                "toolResult": {
                    "toolUseId": tool_id,
                    "content": [{"text": result_summary}]
                }
            }
        ]
    }

    # Send tool result back to the model
    final_output = bedrock.converse(
        modelId=MODEL_ID,
        messages=[user_input, assistant_reply, tool_response_msg],
        toolConfig={  
            "tools": [math_tool],
            "toolChoice": {"auto": {}}
        },
        inferenceConfig={"temperature": 0.7}
    )

    # Display the final response from the assistant
    final_message = final_output["output"]["message"]["content"][0]["text"]
    print("\n=== Final Assistant Response ===")
    print(final_message)

{'toolUse': {'toolUseId': 'tooluse_GcYPqwxmTxCeGQJJuGTWXw', 'name': 'calculateNumbers', 'input': {'num1': 100, 'operation': 'subtract', 'num2': 60}}}
→ Assistant triggered tool: calculateNumbers with input: {'num1': 100, 'operation': 'subtract', 'num2': 60}
← Lambda Function output: {'result': 40.0}

=== Final Assistant Response ===
The outcome of subtracting 60 from 100 is 40.0.
